<a href="https://colab.research.google.com/github/WilyLynx/Piekna_corona_python/blob/master/Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pakiety, metody i inne super-moce



## Pandas
![Panda](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR1bFV2PWd3lpA3UwLk3tNCXO5vaAS-Rx2iFh8Xi1nOsVjaJaYy&usqp=CAU)

`import Pandas as pd`

Pandas jest pakietem umożliwiającym pracę z ramkami danych.


### Wczytanie pakietów

In [0]:
import pandas as pd
import numpy as np

### Wczytanie danych
Dzisiaj będziemy przeglądać dane dotyczące kosaćców


In [0]:
data = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
data.head()

### Ciekawostka dotycząca cech, które tutaj występują
- sepal - działka kielicha
- petal - płatek
![kwiatek](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Mature_flower_diagram.svg/1200px-Mature_flower_diagram.svg.png)

In [0]:
# Policzmy ile jest obserwacji
data.count()

In [0]:
# Dostęp do poszczególnych wartości
# print(data[0][0]) # Coś takiego nie działa, to nie jest tablica 2D
print(data['sepal.length'][0]) # pojedynczy element
print()
print(data['sepal.length']) # cała kolumna

### Wybieranie kolumn
Czasami nie są nam potrzebne wszystkie kolumny. 

In [0]:
sepal_width = data['sepal.width']
sepal_width.head()

In [0]:
sepal_df = data[['sepal.width', 'sepal.length', 'variety']]
sepal_df.head()

### Podstawowa statystyka

In [0]:
data.min() # minimum dla każdej cechy

In [0]:
data['petal.length'].min() # minimum dla jednej cechy

**Zadanie 1**

Dla każdej cechy znaleźć medianę, maksimum oraz średnią wartość.
Ile gatunków kosaćców występuje w zbiorze?

In [0]:
# miejsce na rozwiązanie

### Grupowanie
Czasami nasze obserwacje chcemy podzielić na grupy, np. po gatunku. Następnie możemy użyć wcześniej poznanych metod statystycznych.

In [0]:
data.groupby('variety').size()

In [0]:
data.groupby('variety').min()

## Webscrapping - requests + beautifulsoup4

### Wczytywanie pakietów

In [0]:
import requests
from bs4 import BeautifulSoup

### Czym jest web scrapping ?
Niektóre firmy udostępniają swoje dane w przystępny sposób, np w postaci serwisów REST. Poniżej mamy przykład w jaki sposób GitHub odpowiada na pytanie o repozytorium projektu.

In [0]:
requests.request(url='https://api.github.com/repos/WilyLynx/Piekna_corona_python', method='GET').content

Czasami jednak nie jest tak kolorowo. Czasami dane są prezentowane, ale jedynie w postaci np. wpisów na stronie. Pozyskiwanie danych tego typu nazywamy - web scrapping.

Dzisiaj postaramy się wyciągnąć dane dotyczące zachorowań na COVID-19 w powiecie mińskim.


### Dane dla powiatu mińskiego

In [0]:
site = requests.request('GET', 'http://minsk.psse.waw.pl/4786').content
print(site)

Obserwacja nr 1 - strona to tak na prawdę długi tekst

Obserwacja nr 2 - ten tekst ma specyficzny układ

Wchodzimy na stronę sanepidu i wciskamy F12

Pakiet BeautifulSoup ułatwi nam wyszukiwanie elementów w stronie.

In [0]:
soup = BeautifulSoup(site, 'html.parser')

[Dokumentacja projektu](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

Obserwacja nr 3 - wszystkie interesujące nas dane znajdują się wewnątrz znacznika `<div class='article_inside'>`. Znajdźmy ten znacznik w stronie i sprawdźmy, czy jest tylko jeden taki znacznik.

In [0]:
len(soup.find_all('div', {'class':'article_inside'}))

Zobaczmy teraz co jest w średku.

In [0]:
articles = soup.find_all('div', {'class':'article_inside'})[0]
for article in articles.find_all('div', {'class':'ce_text'}):
  print(article)

Całkiem nieźle, ale miejscami pojawiają się jakieś komunikaty dla podróżnych, które nie bardzo nas interesują. Musimy dodać małe filtrowanie

In [0]:
filer_text = 'powiatu mińskiego związaną z koronawirusem'
articles = soup.find_all('div', {'class':'article_inside'})[0]
messages = articles.find_all('div', {'class':'ce_text'})
filtered_messages = [msg for msg in messages if filer_text in str(msg)]
print(*filtered_messages, sep='\n')

Na przykładzie pierwszego wpisu spróbujmy dojść do interesujących nas danych. Interesuje nas:
- data
- kwarantanna domowa
- hospitalizowani podejrzani
- potwierdzone przypadki
- zgony
- ozdrowieńcy

Dane będziemy przechowywać w prostej liście

In [0]:
first_msg = filtered_messages[0]

data = [] # lista na wszystkie wiadomości
msg_data =[] # lista na wartości dla jednej wiadomości
msg_data.append(first_msg.contents[2].contents[0].contents[0].contents[0][:10]) # pierwsze 10 znaków
print(msg_data)

In [0]:

for ul in first_msg.find_all('ul'):
  for li in ul.find_all('li'):
    print(li)

In [0]:
rows = first_msg.find('ul').find_all('li')
msg_data.append(int([r for r in rows if 'kwarantanną' in str(r)][0].contents[3].contents[0]))
msg_data.append(int([r for r in rows if 'hospitalizowanych ' in str(r)][0].contents[3].contents[0]))
msg_data.append(int([r for r in rows if 'potwierdzonych' in str(r)][0].contents[3].contents[0]))
msg_data.append(int([r for r in rows if 'ozdrowieńców' in str(r)][0].contents[3].contents[0]))

data.append(msg_data)
print(data)

Teraz stworzymy sobie naszą własną ramkę danych do łatwiejszego przechowywania.

In [0]:
minsk_df = pd.DataFrame(data, columns = ['data', 'kwarantanna', 'hospitalizowani', 'potwierdzeni', 'ozdrowieńcy'])
minsk_df.head()

Chyba wszystko działa. Spróbujmy teraz powtórzyć to dla wszystkich wiadomości

In [0]:
data = [] # lista na wszystkie wiadomości
for msg in filtered_messages:
  try: # czasami się coś nie udaje wtedy pojawia się błąd (wyjątek)
    msg_data =[]
    # data
    msg_data.append(msg.contents[2].contents[0].contents[0].contents[0][:10]) # pierwsze 10 znaków
    # cechy
    rows = msg.find('ul').find_all('li')
    msg_data.append(int([r for r in rows if 'kwarantanną' in str(r)][0].contents[3].contents[0]))
    msg_data.append(int([r for r in rows if 'hospitalizowanych ' in str(r)][0].contents[3].contents[0]))
    msg_data.append(int([r for r in rows if 'potwierdzonych' in str(r)][0].contents[3].contents[0]))
    msg_data.append(int([r for r in rows if 'ozdrowieńców' in str(r)][0].contents[3].contents[0]))

    data.append(msg_data)
  except:
    pass # udajemy że nie widzimy błędu i lecimy dalej

minsk_df = pd.DataFrame(data, columns = ['data', 'kwarantanna', 'hospitalizowani', 'potwierdzeni', 'ozdrowieńcy'])
minsk_df.loc[:, 'data'] = pd.to_datetime(minsk_df['data'])
minsk_df = minsk_df.set_index('data')
minsk_df

Coś mało tych danych. Jednak musimy się bardziej postarać. Wygląda na to, że jest problem z datą. Musimy to zrobić w inny sposób. Spróbujemy wyciągnąć datę z nagłówka.

In [0]:
import re
match = re.search('\d{2}\.\d{2}\.2020','Komunikat Państwowego Powiatowego Inspektora Sanitarnego w Mińsku Mazowieckim  z dnia 19.04.2020 r. przeds')
match.group(0)

In [0]:
import re
data = [] # lista na wszystkie wiadomości
for msg in filtered_messages:
  try:
    msg_data =[]
    # data
    if len(msg.find_all('h3')) > 0:
      header = msg.find_all('h3')[0]
    else:
      header = msg.find_all('em')[0]
    match = re.search('\d{2}\.\d{2}\.2020',str(header))
    date = match.group(0)
    msg_data.append(date)

    # cechy
    rows = msg.find('ul').find_all('li')
    try:
      msg_data.append(int([r for r in rows if 'kwarantanną' in str(r)][0].contents[3].contents[0]))
    except Exception:
      msg_data.append(None)

    try:
      msg_data.append(int([r for r in rows if 'hospitalizowanych ' in str(r)][0].contents[3].contents[0]))
    except Exception:
      msg_data.append(None)

    try:
      msg_data.append(int([r for r in rows if 'potwierdzonych' in str(r)][0].contents[3].contents[0]))
    except Exception:
      msg_data.append(None)

    try:
      msg_data.append(int([r for r in rows if 'ozdrowieńców' in str(r)][0].contents[3].contents[0]))
    except Exception:
      msg_data.append(None)

    data.append(msg_data)
  except Exception:
    pass

minsk_df = pd.DataFrame(data, columns = ['data', 'kwarantanna', 'hospitalizowani', 'potwierdzeni', 'ozdrowieńcy'])
minsk_df.loc[:, 'data'] = pd.to_datetime(minsk_df['data'],format='%d.%m.%Y')
minsk_df = minsk_df.set_index('data')
minsk_df

Zobaczmy jak zmieniała się liczba osób w kwarantannie

In [0]:
import matplotlib.pyplot as plt
# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

minsk_df['kwarantanna'].plot(linewidth=0.5);